### Text Prediction

In [1]:
import os
import nltk
from nltk.util import ngrams
import sqlite3

In [2]:
base_file = open("data/1.txt", 'rt')
raw_text = base_file.read()
base_file.close()

token_list1 = nltk.word_tokenize(raw_text)
token_list2 = [word.replace("'", "") for word in token_list1]
token_list3 = list(filter(lambda token: nltk.tokenize.punkt.PunktToken(token).is_non_punct, token_list2))
token_list4=[word.lower() for word in token_list3]

print("\nTotal Tokens : ",len(token_list4))


Total Tokens :  579


In [3]:
conn = sqlite3.connect(":memory:")
conn.execute('''DROP TABLE IF EXISTS NGRAMS''')
conn.execute('''CREATE TABLE NGRAMS 
         (FIRST   TEXT  NOT NULL,
          SECOND  TEXT  NOT NULL,
          COUNTS  INT   NOT NULL,
         CONSTRAINT PK_GRAMS PRIMARY KEY (FIRST,SECOND));''')

for i in ngrams(token_list4,2):
    insert_str="INSERT INTO NGRAMS (FIRST,SECOND,COUNTS) \
          VALUES ('" + i[0] + "','" + i[1] + "',1 ) \
          ON CONFLICT(FIRST,SECOND) DO UPDATE SET COUNTS=COUNTS + 1"   
    conn.execute(insert_str);

cursor = conn.execute("SELECT FIRST, SECOND, COUNTS from NGRAMS LIMIT 5")
for gram_row in cursor:
    print("FIRST =", gram_row[0], "; SECOND =",gram_row[1],"; COUNT =",gram_row[2])

FIRST = in ; SECOND = this ; COUNT = 2
FIRST = this ; SECOND = practical ; COUNT = 1
FIRST = practical ; SECOND = hands-on ; COUNT = 1
FIRST = hands-on ; SECOND = course ; COUNT = 1
FIRST = course ; SECOND = learn ; COUNT = 1


In [4]:
def recommend(str):
    nextwords = []
    cur_filter = conn.execute("SELECT SECOND from NGRAMS \
                              WHERE FIRST='" + str + "' \
                              ORDER BY COUNTS DESC")
    for filt_row in cur_filter:
        nextwords.append(filt_row[0])
    return nextwords

print("Next word for data :", recommend("data"))
print("\nNext word for science :", recommend("science"))

Next word for data : ['science', 'analysis', 'data', 'from', 'in', 'mining', 'munging', 'node.js', 'preparation', 'scientists', 'visualization', 'you']

Next word for science : ['begins', 'requires', 'specialists', 'teams']
